Good Reads

Dataset Description
This dataset contains more than 1.3M book reviews about 25,475 books and 18,892 users , which is a review subset for spoiler detection, where each book/user has at least one associated spoiler review.

Goodreads Books Review Rating Prediction
Reviews are a good way to judge the quality of any product, whether it's books, clothes, technology, or anything else. When you want to buy something online these days, the first thing that comes to mind is the reviews from past buyers and the overall rating the product has received.
Reader feedback, whether positive or negative, five stars or one star, will encourage the product owner to make improvements.
Reader connection and engagement will be encouraged by book reviews, whether they be left on Amazon, Goodreads, or social media. Readers must determine whether or not other readers are enjoying the book.

In this competition you will work with a challenging dataset consisting reviews from the Goodreads book review website, and a variety of attributes describing the items. and you have to predict review rating which ranges from 0 to 5.

In [79]:
#pip install nltk
#pip install spacy
# pip install textblob

SyntaxError: invalid syntax (3177122387.py, line 2)

In [151]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, f1_score
from textblob import TextBlob #sentiment anlysis library
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer #another sentiment analysis library
#import spacy #another sentiment analysis library
from sklearn.feature_extraction.text import TfidfVectorizer #the following 4 libraries are for a customize model of sentiment with sklearn
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
from keras.preprocessing.text import Tokenizer #the following 4 libraries are for a customize model of sentiment with deep learning: keras
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU
from keras.utils.np_utils import to_categorical
import tensorflow as tf # This library is for a customize model of sentiement with deep learning: Tensor flow
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense


In [94]:
# Load the data
path = ".\goodreads-books-reviews-290312\goodreads_train_sample.csv"
df = pd.read_csv(path)
print (df.shape)
df.head()

(90000, 12)


,Unnamed: 0,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,898402,ed556b92506c3452b42fffed31697a1a,25125233,182718faad99666b70f73f3b7ffbbdb7,3,Jessica Broussard has only ever had her father...,Thu Nov 19 07:48:55 -0800 2015,Thu Nov 19 11:31:55 -0800 2015,Wed Nov 18 00:00:00 -0800 2015,Tue Nov 17 00:00:00 -0800 2015,0,0
1,853518,fcf6bca39e8f5333ba018b0e146ccfec,6837103,228c47ca18ed4c1598ae3c9214530b6b,5,"Set in the late 1700s and early 1800s, the sto...",Tue Dec 06 13:19:29 -0800 2011,Mon Dec 19 11:28:19 -0800 2011,Mon Dec 19 00:00:00 -0800 2011,Tue Dec 06 00:00:00 -0800 2011,0,0
2,366741,b8f6f163c2161555c6d887632b2ff4a2,17948485,6137b1fe0159b7eaa56a04293a00fd49,5,This book is the bomb! Love every single page ...,Fri Jul 26 17:43:19 -0700 2013,Sat Aug 10 20:06:19 -0700 2013,Sat Jul 27 00:00:00 -0700 2013,Fri Jul 26 00:00:00 -0700 2013,0,0
3,476233,33162c8e64b16bcbddc9808f3c716342,18405,c5b3dc0c0416d850380d80f5304be91f,5,Wherein I attempt to write a review using all ...,Wed Jun 30 08:01:44 -0700 2010,Tue Dec 31 06:07:21 -0800 2013,Fri Feb 18 00:00:00 -0800 2011,Sat Feb 12 00:00:00 -0800 2011,46,14
4,856723,37d8353490e210e2b3766336be99ebd4,26218626,234b51de9a79dfd51b5dc2b48df972ec,3,I can't believe I have only one volume left. T...,Wed Jun 29 19:48:42 -0700 2016,Thu Mar 30 07:18:31 -0700 2017,Thu Mar 30 07:18:31 -0700 2017,Thu Mar 30 00:00:00 -0700 2017,0,0


In [95]:
df = df.drop(["Unnamed: 0"], axis=1) #this just when I upload the sample of 10% of data

In [96]:
#Check type var
df.dtypes

user_id         object
book_id          int64
review_id       object
rating           int64
review_text     object
date_added      object
date_updated    object
read_at         object
started_at      object
n_votes          int64
n_comments       int64
dtype: object

In [97]:
df.count()

user_id         90000
book_id         90000
review_id       90000
rating          90000
review_text     90000
date_added      90000
date_updated    90000
read_at         80742
started_at      62664
n_votes         90000
n_comments      90000
dtype: int64

In [98]:
df.isna().sum(axis = 0)

user_id             0
book_id             0
review_id           0
rating              0
review_text         0
date_added          0
date_updated        0
read_at          9258
started_at      27336
n_votes             0
n_comments          0
dtype: int64

In [99]:
df = df.dropna()

In [100]:
df.describe(include='all')

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
count,61662,6.166200e+04,61662,61662.000000,61662,61662,61662,61662,61662,61662.000000,61662.000000
unique,8906,NaN,61662,NaN,61381,61641,61513,28535,6242,NaN,NaN
top,843a44e2499ba9362b47a089b0b0ce75,NaN,182718faad99666b70f73f3b7ffbbdb7,NaN,Review to come.,Sat Jul 29 09:26:28 -0700 2017,Fri Jul 19 06:15:59 -0700 2013,Fri Jan 01 00:00:00 -0800 2016,Sun Jan 01 00:00:00 -0800 2017,NaN,NaN
freq,173,NaN,1,NaN,43,2,4,48,80,NaN,NaN
mean,NaN,1.457580e+07,NaN,3.793876,NaN,NaN,NaN,NaN,NaN,3.631329,1.152833
std,NaN,9.167256e+06,NaN,1.128300,NaN,NaN,NaN,NaN,NaN,15.489702,5.975086
min,NaN,1.000000e+00,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
25%,NaN,7.746506e+06,NaN,3.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
50%,NaN,1.576785e+07,NaN,4.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
75%,NaN,2.185739e+07,NaN,5.000000,NaN,NaN,NaN,NaN,NaN,2.000000,0.000000


Sentiment analysis 1: Textblob
In this code, we use the pandas library to read the book review data into a dataframe, and the TextBlob library to perform sentiment analysis. We apply the sentiment analysis to each review in the "review" column of the dataframe and store the results in a new column called "sentiment". The polarity score of each review ranges from -1 (most negative) to 1 (most positive).

In [101]:
# Create a new column in the dataframe to store the sentiment scores
df['review_text_textblob'] = df['review_text'].apply(lambda x: TextBlob(x).sentiment.polarity) #it takes 14 minutes with the completed data

Sentiment Analysis 2: Nltk
In this code, we use the pandas library to read the book review data into a dataframe, and the SentimentIntensityAnalyzer from the nltk library to perform sentiment analysis. We apply the sentiment analysis to each review in the "review" column of the dataframe and store the results in a new column called "sentiment". The polarity score of each review ranges from -1 (most negative) to 1 (most positive).

In [102]:
# Initialize the SentimentIntensityAnalyzer
nltk.download("vader_lexicon")
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\edofa\AppData\Roaming\nltk_data...


In [103]:
# Create a new column in the dataframe to store the sentiment scores
df['review_text_nltk'] = df['review_text'].apply(lambda x: sia.polarity_scores(x)['compound']) #it takes 2 min with the sample data

Sentiment Analysis 3: Spacy
In this code, we use the pandas library to read the book review data into a dataframe, and the spacy library to perform sentiment analysis. We apply the sentiment analysis to each review in the "review" column of the dataframe and store the results in a new column called "sentiment". The sentiment score of each review is either 1 (positive) or -1 (negative).

In [104]:
# Load the English language model
#nlp = spacy.load("en_core_web_sm")

NameError: name 'spacy' is not defined

In [ ]:
# Create a function to get the sentiment score of a review
#def get_sentiment(review):
    #doc = nlp(review)
    #if doc.cats['POSITIVE'] > doc.cats['NEGATIVE']:
        #return 1
    #else:
        #return -1

In [ ]:
# Create a new column in the dataframe to store the sentiment scores
#df['review_text_spacy'] = df['review_text'].apply(lambda x: get_sentiment(x))

Sentiment analysis 4: Customize model with sklearn

In this code, we use the pandas library to read the book review data into a dataframe and split it into training and testing sets.

Next, we use the TfidfVectorizer from scikit-learn to convert the text data into numerical features using the Tf-idf representation.

We then train a linear support vector machine (SVM) classifier using the LinearSVC class from scikit-learn on the training data.

We use the trained model to make predictions on the test data and evaluate its accuracy using the accuracy_score function from scikit-learn.

Finally, we add the prediction results as a new column in the dataframe using df.loc[] to assign the values only for the rows in the test data.


In [117]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['review_text'], df['rating'], test_size=0.3)

In [118]:
# Convert the text data into numerical features using Tf-idf
vectorizer = TfidfVectorizer()
X_train_features = vectorizer.fit_transform(X_train)
X_test_features = vectorizer.transform(X_test)

In [119]:
# Train a linear support vector machine (SVM) classifier
clf = LinearSVC()
clf.fit(X_train_features, y_train)

LinearSVC()

In [120]:
# Make predictions on the test data
y_pred = clf.predict(X_test_features)

In [121]:
# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.4888912914211579


In [122]:
# Add the prediction results as a new column in the dataframe
df['review_text_sktlearn'] = np.nan
df.loc[X_test.index, 'review_text_sktlearn'] = y_pred

Sentiment analysis 5: Keras

Here's a step-by-step explanation of the process for training a multi-class sentiment analysis model in Keras to predict the rating of a book review:

Load the data: The first step is to load the book review data into a pandas dataframe. In this example, the data is loaded from a CSV file using the read_csv function.

Preprocess the text data: The next step is to preprocess the text data. To do this, we use the Tokenizer class from Keras. We first fit the tokenizer on the text data, which creates a vocabulary of the most common words in the text. Then, we use the tokenizer to convert the text data into numerical sequences, where each word is represented by its index in the vocabulary. Finally, we use the pad_sequences function from Keras to pad the sequences to a fixed length of 100.

Convert the rating data into categorical labels: We convert the rating data into categorical labels using the to_categorical function from Keras. This function converts the rating data into a one-hot encoded representation, where each rating is represented as a vector of 6 elements with a 1 in the index corresponding to the rating and 0s in all other elements.

Split the data into training and testing sets: We split the preprocessed data into training and testing sets using the train_test_split function from scikit-learn. This function splits the data into two sets, one for training the model and one for evaluating its performance.

Define the model architecture: We define the model architecture using the Sequential model from Keras. We add an Embedding layer, which maps the word indices to dense vectors of fixed size. The dense vectors are used as input to the next layer. We add a GRU (Gated Recurrent Unit) layer, which is a type of recurrent neural network that processes sequences of input data. The GRU layer allows the model to take into account the context and dependencies between words in the review. Finally, we add a Dense layer with a softmax activation function, which outputs the probability of each rating class.

Compile the model: We compile the model using the compile method and specify the loss function, optimizer, and evaluation metrics. The loss function measures the difference between the predicted and actual ratings. The optimizer updates the model weights to minimize the loss. The evaluation metrics measure the performance of the model.

Train the model: We train the model using the fit method. The model is trained on the training data, and the model weights are updated to minimize the loss function. The model is trained for 10 epochs, where an epoch is one pass over the training data. The batch size is set to 32, which means that the model updates its weights after processing 32 reviews at a time.

Evaluate the model: Finally, we evaluate the model on the test data using the evaluate method. The evaluate method returns the loss and accuracy of the model on the test data.

By following these steps, we can train a multi-class sentiment analysis model in Keras to predict the rating of a book review based on its text.


In [124]:
# Preprocess the text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df['review_text'])
X = tokenizer.texts_to_sequences(df['review_text'])
X = pad_sequences(X, maxlen=100)

In [127]:
# Convert the rating data into categorical labels
y = to_categorical(df['rating'], num_classes=6)

In [128]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [129]:
# Define the model architecture
model = Sequential()
model.add(Embedding(5000, 128, input_length=100))
model.add(GRU(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(6, activation='softmax'))

In [130]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [131]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test)) #it takes 22 min for the sample data


Epoch 1/10
1349/1349 [==============================] - 131s 95ms/step - loss: 1.2230 - accuracy: 0.4669 - val_loss: 1.0902 - val_accuracy: 0.5312
Epoch 2/10
1349/1349 [==============================] - 132s 98ms/step - loss: 1.0447 - accuracy: 0.5562 - val_loss: 1.0743 - val_accuracy: 0.5379
Epoch 3/10
1349/1349 [==============================] - 135s 100ms/step - loss: 0.9679 - accuracy: 0.5931 - val_loss: 1.0857 - val_accuracy: 0.5396
Epoch 4/10
1349/1349 [==============================] - 139s 103ms/step - loss: 0.8993 - accuracy: 0.6259 - val_loss: 1.1321 - val_accuracy: 0.5247
Epoch 5/10
1349/1349 [==============================] - 143s 106ms/step - loss: 0.8331 - accuracy: 0.6562 - val_loss: 1.1696 - val_accuracy: 0.5180
Epoch 6/10
1349/1349 [==============================] - 140s 104ms/step - loss: 0.7704 - accuracy: 0.6850 - val_loss: 1.2254 - val_accuracy: 0.5040
Epoch 7/10
1349/1349 [==============================] - 137s 101ms/step - loss: 0.7092 - accuracy: 0.7139 - val_lo

In [134]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc)

579/579 [==============================] - 7s 12ms/step - loss: 1.6212 - accuracy: 0.4842
Test accuracy: 0.48418834805488586


It can be observed that the accuracy does not improbe to much regard to sklearn model

In [135]:
# Create a predict function
def predict(model, x_test):
    predictions = model.predict(x_test)
    predicted_classes = np.argmax(predictions, axis=1)
    return predicted_classes

In [139]:
# Predict the sentiment for each review
y_pred = predict(model, X)

In [140]:
# Create a new column in the dataframe with the predicted sentiments
df['review_text_keras'] = y_pred


Sentiment analysis 5: Tensorflow
here's a more detailed explanation of the TensorFlow model:

1:Import the TensorFlow library:
2:Define the model architecture using the Sequential model:
This code creates a model with 3 layers:

The Embedding layer takes the input data, which is a numerical representation of the book reviews, and maps each word to a dense vector of fixed size. The input_dim parameter specifies the size of the vocabulary (i.e. the number of unique words in the data), output_dim is the size of the dense vector representation, and input_length is the maximum length of the reviews.

The LSTM (Long-Short Term Memory) layer is a type of recurrent neural network layer that is commonly used for sequence data, such as text data. This layer processes the sequences and generates an output representation that summarizes the input sequence. The number 32 represents the number of units in the LSTM layer.

The Dense layer is a fully connected layer that takes the output of the LSTM layer and produces 6 output units, each corresponding to a rating class. The softmax activation function is used to ensure that the outputs are probabilities that sum up to 1, so that they can be interpreted as class probabilities.
3: Compile the model:
The compile method configures the model for training. The loss function is set to sparse_categorical_crossentropy, which is a loss function that is suitable for multi-class classification problems. The optimizer is set to adam, which is a popular optimization algorithm for deep learning models. The metrics argument is set to accuracy, which tells the model to report the accuracy of the predictions during training.
4: Train the model:
The fit method trains the model on the training data X_train and y_train. The epochs parameter specifies the number of times the model will see the entire training data (10 in this case). The batch_size parameter specifies the number of samples per gradient update (32 in this case). The validation_data argument provides the validation data X_val and y_val to evaluate the model after each epoch.

Once the model is trained, the predictions can be obtained using the predict method, as explained in the previous answer

In [152]:
# Preprocess the data
reviews = df['review_text'].values
labels = df['rating'].values

# Calculate the vocab size
vocab = set()
for review in reviews:
    words = review.split()
    for word in words:
        vocab.add(word)
vocab_size = len(vocab)

In a natural language processing (NLP) task, the vocab_size is the size of the vocabulary used in the input text data. The vocabulary is a set of unique words in the input text. In the case of sentiment analysis, vocab_size is the number of unique words in the input reviews.

In the code, vocab_size is used as the input dimension of the Embedding layer in the neural network model. The Embedding layer is used to convert the input words into a dense vector representation, which can be fed into other layers in the network.

To define the vocab_size, you need to calculate the size of the vocabulary in the input text data.

In this code, reviews is a list of the reviews in the dataframe, and vocab is a set that contains all the unique words in the reviews. The size of vocab is calculated using the len() function, and it is stored in vocab_size.

In [147]:
# Calculate the max length
max_length = 0
for review in reviews:
    words = review.split()
    length = len(words)
    if length > max_length:
        max_length = length
print(length)

In a natural language processing (NLP) task, max_length refers to the maximum length of the input sequences, such as sentences or documents. In the case of sentiment analysis, max_length is the maximum number of words in a review.

The max_length is used to pad or truncate the input sequences to a fixed length, so that all the input sequences have the same length and can be processed by the neural network.

To define the max_length, you need to calculate the length of the longest review in your data.

In this code, reviews is a list of the reviews in the dataframe, and max_length is initialized to 0. The length of each review is calculated using the len() function, and the longest review is stored in max_length.

In [153]:
# Tokenize the reviews
tokenizer = Tokenizer()
tokenizer.fit_on_texts(reviews)
reviews = tokenizer.texts_to_sequences(reviews)

In [154]:
# Pad or truncate the reviews to have a length of 100
reviews = pad_sequences(reviews, maxlen=100)

In [155]:
## Define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1,
                    output_dim=100,
                    input_length=100))
model.add(LSTM(64))
model.add(Dense(6, activation='softmax'))

In [156]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [157]:
# Fit the model
history = model.fit(reviews, labels, epochs=10, batch_size=32)

Epoch 1/10
1927/1927 [==============================] - 233s 120ms/step - loss: 1.2153 - accuracy: 0.4701
Epoch 2/10
1927/1927 [==============================] - 238s 123ms/step - loss: 0.9979 - accuracy: 0.5846
Epoch 3/10
1927/1927 [==============================] - 236s 122ms/step - loss: 0.8250 - accuracy: 0.6698
Epoch 4/10
1927/1927 [==============================] - 229s 119ms/step - loss: 0.6455 - accuracy: 0.7524
Epoch 5/10
1927/1927 [==============================] - 219s 114ms/step - loss: 0.4816 - accuracy: 0.8201
Epoch 6/10
1927/1927 [==============================] - 218s 113ms/step - loss: 0.3494 - accuracy: 0.8721
Epoch 7/10
1927/1927 [==============================] - 218s 113ms/step - loss: 0.2623 - accuracy: 0.9052
Epoch 8/10
1927/1927 [==============================] - 217s 113ms/step - loss: 0.1977 - accuracy: 0.9284
Epoch 9/10
1927/1927 [==============================] - 219s 114ms/step - loss: 0.1552 - accuracy: 0.9449
Epoch 10/10
1927/1927 [=======================

In this code, the input reviews are first tokenized using the Tokenizer class from the tensorflow.keras.preprocessing.text module. The reviews are then converted to sequences of integers using the texts_to_sequences method. Finally, the input sequences are padded or truncated to have a length of 100 using the pad_sequences function.

The model consists of an Embedding layer, an LSTM layer, and a Dense layer with 6 units and a softmax activation function. The model is compiled using the adam optimizer, sparse_categorical_crossentropy loss function, and accuracy as the evaluation metric. Finally, the model is trained on the input data for 10 epochs with a batch size of 32.

In [158]:
# Use the model to make predictions
predictions = model.predict(reviews)

# Convert the predictions to class labels
predicted_classes = tf.argmax(predictions, axis=1)

# Add the predictions to a new column in the dataframe
df['review_text_tf'] = predicted_classes.numpy()


Note: Keras is a high-level API for building deep learning models that runs on top of TensorFlow.

When you build a deep learning model using Keras, you are actually building a TensorFlow model, but with a more user-friendly API. In other words, Keras provides a simpler way to create TensorFlow models. This is why the code mentioned in my previous answer uses the tf.keras API, which is a TensorFlow implementation of the Keras API.

In [159]:
# Now we check out all the sentiment models into de dataset
df.head()

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments,review_text_textblob,review_text_nltk,review_text_sktlearn,review_text_keras,review_text_tf
0,ed556b92506c3452b42fffed31697a1a,25125233,182718faad99666b70f73f3b7ffbbdb7,3,Jessica Broussard has only ever had her father...,Thu Nov 19 07:48:55 -0800 2015,Thu Nov 19 11:31:55 -0800 2015,Wed Nov 18 00:00:00 -0800 2015,Tue Nov 17 00:00:00 -0800 2015,0,0,0.143082,0.9964,NaN,3,4
1,fcf6bca39e8f5333ba018b0e146ccfec,6837103,228c47ca18ed4c1598ae3c9214530b6b,5,"Set in the late 1700s and early 1800s, the sto...",Tue Dec 06 13:19:29 -0800 2011,Mon Dec 19 11:28:19 -0800 2011,Mon Dec 19 00:00:00 -0800 2011,Tue Dec 06 00:00:00 -0800 2011,0,0,0.101091,0.8885,4.0,5,5
2,b8f6f163c2161555c6d887632b2ff4a2,17948485,6137b1fe0159b7eaa56a04293a00fd49,5,This book is the bomb! Love every single page ...,Fri Jul 26 17:43:19 -0700 2013,Sat Aug 10 20:06:19 -0700 2013,Sat Jul 27 00:00:00 -0700 2013,Fri Jul 26 00:00:00 -0700 2013,0,0,0.203858,0.9961,NaN,5,5
3,33162c8e64b16bcbddc9808f3c716342,18405,c5b3dc0c0416d850380d80f5304be91f,5,Wherein I attempt to write a review using all ...,Wed Jun 30 08:01:44 -0700 2010,Tue Dec 31 06:07:21 -0800 2013,Fri Feb 18 00:00:00 -0800 2011,Sat Feb 12 00:00:00 -0800 2011,46,14,0.079210,0.9686,4.0,5,5
4,37d8353490e210e2b3766336be99ebd4,26218626,234b51de9a79dfd51b5dc2b48df972ec,3,I can't believe I have only one volume left. T...,Wed Jun 29 19:48:42 -0700 2016,Thu Mar 30 07:18:31 -0700 2017,Thu Mar 30 07:18:31 -0700 2017,Thu Mar 30 00:00:00 -0700 2017,0,0,-0.058333,-0.5863,NaN,3,2


In [190]:
#Matriz de correlaciones
corrMatrix = df.corr() #Genereamos una matriz de correlaciones entre todas las variables
corrMatrix.style.background_gradient(cmap='coolwarm')

,user_id,book_id,review_id,rating,n_votes,n_comments,review_text_textblob,review_text_nltk,review_text_sktlearn,review_text_keras,review_text_tf
user_id,1.000000,-0.007307,0.009213,-0.008762,-0.000984,0.000615,-0.000921,-0.004950,-0.009048,-0.001957,-0.007241
book_id,-0.007307,1.000000,-0.006508,0.031278,0.093277,0.034402,0.018430,0.049152,0.033896,0.028850,0.032204
review_id,0.009213,-0.006508,1.000000,-0.008225,-0.003161,-0.005827,0.000247,0.001723,-0.016075,-0.012064,-0.012693
rating,-0.008762,0.031278,-0.008225,1.000000,0.024718,0.018426,0.296344,0.254273,0.529924,0.786581,0.969597
n_votes,-0.000984,0.093277,-0.003161,0.024718,1.000000,0.669298,-0.035070,0.016577,0.034834,0.025871,0.020630
n_comments,0.000615,0.034402,-0.005827,0.018426,0.669298,1.000000,-0.042743,0.010405,0.021635,0.019113,0.014689
review_text_textblob,-0.000921,0.018430,0.000247,0.296344,-0.035070,-0.042743,1.000000,0.390609,0.287534,0.306677,0.301290
review_text_nltk,-0.004950,0.049152,0.001723,0.254273,0.016577,0.010405,0.390609,1.000000,0.213159,0.252493,0.254078
review_text_sktlearn,-0.009048,0.033896,-0.016075,0.529924,0.034834,0.021635,0.287534,0.213159,1.000000,0.550911,0.541941
review_text_keras,-0.001957,0.028850,-0.012064,0.786581,0.025871,0.019113,0.306677,0.252493,0.550911,1.000000,0.794006


Here we have 5 model of sentiment analysis, textblob and nltk are independent of rating, and sklearn, keras and tf are dependent of rating. The regression is high but we have to consider it because the 70% of the data was used for train. So it could be overfitting. For example for Keras the test data just gave us 50% de accuracy but if we use the whole data (train+test) it gives us almost 79%. The best results is by far given by Tensor flow with 97%.

In [174]:
df = df.dropna()
print(df.shape)

(18499, 16)


In [175]:
# Encoding the string variables
labelencoder = LabelEncoder()
df['user_id'] = labelencoder.fit_transform(df['user_id'])
df['book_id'] = labelencoder.fit_transform(df['book_id'])
df['review_id'] = labelencoder.fit_transform(df['review_id'])

In [176]:
# Extract features and target variable
X = df.drop(['book_id', 'review_id', 'user_id',"rating",'date_added','date_updated','read_at','started_at', 'review_text'], axis=1)
y = df["rating"]

In [177]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [178]:
# Fit a linear regression model to the training data
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [179]:
# Predict the star ratings on the test data
y_pred = model.predict(X_test)

In [180]:
# Round the predicted values to the nearest integer to obtain class labels
y_pred_class = np.round(y_pred)

In [181]:
# Calculate the F1 score
f1 = f1_score(y_test, y_pred_class, average='weighted')
print("F1 Score:", f1)

F1 Score: 0.9745864635125381


In [182]:
# Check if the F1 score is over 70%
if f1 > 0.7:
    print("The model has a good F1 score.")
else:
    print("The model needs improvement. Try using a different model or adding more features.")

The model has a good F1 score.


Modelo 2 KNN

In [183]:
# Train a KNN model on the training data
knn = KNeighborsRegressor(n_neighbors=50)
knn.fit(X_train, y_train)

# Predict the ratings on the test data
y_pred_knn = knn.predict(X_test)

In [184]:
# Calculate the MSE and R-squared for the KNN model
mse_knn = mean_squared_error(y_test, y_pred_knn)
r2_knn = r2_score(y_test, y_pred_knn)
print("KNN Model")
print("MSE: ", mse_knn)
print("R-squared: ", r2_knn)

KNN Model
MSE:  0.14968594594594595
R-squared:  0.8797989148822649


For the MSE, a smaller value indicates a better fit of the model to the data. A value of 0 means the model perfectly predicts the target. However, the MSE is sensitive to outliers, so it may not always be a reliable measure of the model's performance.

For the R-squared, the value ranges from 0 to 1, with higher values indicating a better fit of the model to the data. A value of 1 means the model perfectly predicts the target. However, having a high R-squared does not always mean that the model is a good fit, as it can also increase with the addition of irrelevant variables

In [185]:
# Perform cross-validation on the KNN model
cv_scores_knn = cross_val_score(knn, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
cv_scores_knn = np.abs(cv_scores_knn)
print("Cross-Validation MSE: ", cv_scores_knn.mean())

Cross-Validation MSE:  0.18030843646045502


Model 3: Random Forest

In [186]:
# Train a Random Forest model on the training data
rf = RandomForestRegressor(n_estimators=100, random_state=0)
rf.fit(X_train, y_train) #it takes 9 minutes with the completed data


RandomForestRegressor(random_state=0)

In [187]:
# Predict the ratings on the test data
y_pred_rf = rf.predict(X_test)

In [188]:
# Calculate the MSE and R-squared for the Random Forest model
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)
print("Random Forest Model")
print("MSE: ", mse_rf)
print("R-squared: ", r2_rf)

Random Forest Model
MSE:  0.07651076203110324
R-squared:  0.9385601863875418


To the random forest is slightly better performance but still far away from a satisfying results

In [189]:
# Perform cross-validation on the Random Forest model
cv_scores_rf = cross_val_score(rf, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
cv_scores_rf = np.abs(cv_scores_rf)
print("Cross-Validation MSE: ", cv_scores_rf.mean()) ##it takes 35 minutes with the completed data

Cross-Validation MSE:  0.09378429179341084


The lower the Mean Squared Error (MSE) in cross-validation, the better the performance of the model. A good MSE score indicates that the model is making predictions that are close to the true values, while a high MSE score suggests that the model is making significant errors in its predictions.

A MSE of 0 indicates a perfect fit, while a high MSE value suggests a poor fit. However, it is important to keep in mind that our range of working is between 1 to 5.

In general, you can compare the cross-validation MSE of your model with the MSE of a baseline model or with the MSE of other models that you have tried to see if the performance of your model is good or bad.